# 84. 古典的フィルタの解剖学

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

畳み込みの実装方法を学んだ今、**人間が設計した古典的なカーネル**を深く理解します。これらのフィルタは、CNNが「学習する」フィルタの原型であり、画像処理の基礎を形成しています。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口
├── 80. 畳み込みとは何か（直感編）  ✅
├── 81. 畳み込みの数学的定義  ✅
├── 82. NumPyスクラッチ実装（基礎編）  ✅
├── 83. NumPyスクラッチ実装（高速化編）  ✅
├── 84. 古典的フィルタの解剖学  ← 今ここ
└── 85. カーネルの可視化と解釈
```

## 学習目標

1. **平滑化フィルタの仕組みと効果を理解する**（ボックス、ガウシアン）
2. **微分フィルタがエッジを検出する原理を理解する**（Sobel、Laplacian）
3. **複合フィルタを設計できる**（シャープ化、LoG、DoG）
4. **周波数領域でフィルタの意味を解釈できる**

## 目次

1. [フィルタ = 「見たいものを見る眼鏡」](#1-フィルタ--見たいものを見る眼鏡)
2. [平滑化フィルタ（ローパス）](#2-平滑化フィルタローパス)
3. [微分フィルタ（ハイパス）](#3-微分フィルタハイパス)
4. [複合フィルタ](#4-複合フィルタ)
5. [フィルタの周波数領域での解釈](#5-フィルタの周波数領域での解釈)
6. [実験：自分だけのフィルタを設計](#6-実験自分だけのフィルタを設計)

---

## 環境セットアップ

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

np.random.seed(42)
print("環境準備完了")

In [ ]:
# 高速畳み込み関数（前回のノートブックから）

def conv2d(image, kernel, padding='same'):
    """
    2D畳み込み（高速版）
    
    Parameters:
    -----------
    image : ndarray (H, W) or (H, W, C)
        入力画像
    kernel : ndarray (kH, kW)
        カーネル
    padding : 'same', 'valid', or int
        パディング
    
    Returns:
    --------
    output : ndarray
    """
    # カラー画像の場合は各チャネルに適用
    if image.ndim == 3:
        result = np.zeros_like(image, dtype=float)
        for c in range(image.shape[2]):
            result[:, :, c] = conv2d(image[:, :, c], kernel, padding)
        return result
    
    k_h, k_w = kernel.shape
    
    # パディング計算
    if padding == 'same':
        pad_h = (k_h - 1) // 2
        pad_w = (k_w - 1) // 2
    elif padding == 'valid':
        pad_h, pad_w = 0, 0
    else:
        pad_h = pad_w = padding
    
    # パディング適用
    if pad_h > 0 or pad_w > 0:
        image = np.pad(image, ((pad_h, k_h-1-pad_h), (pad_w, k_w-1-pad_w)), 
                      mode='reflect')
    
    # 畳み込み
    windows = sliding_window_view(image, window_shape=(k_h, k_w))
    output = np.sum(windows * kernel, axis=(-2, -1))
    
    return output

print("畳み込み関数を定義しました")

In [ ]:
# テスト画像の作成

def create_test_image(size=128):
    """様々な特徴を含むテスト画像を作成"""
    img = np.zeros((size, size))
    
    # 背景グラデーション
    x = np.linspace(0, 1, size)
    y = np.linspace(0, 1, size)
    X, Y = np.meshgrid(x, y)
    img = 0.1 * X + 0.05 * Y
    
    # 白い四角形（シャープなエッジ）
    img[20:50, 20:50] = 0.9
    
    # 白い円（滑らかなエッジ）
    center = (size*3//4, size//4)
    radius = size // 8
    yy, xx = np.ogrid[:size, :size]
    mask = (xx - center[0])**2 + (yy - center[1])**2 <= radius**2
    img[mask] = 0.8
    
    # 斜めの線
    for i in range(40):
        if 70+i < size and 20+i < size:
            img[70+i, 20+i:23+i] = 0.7
    
    # チェッカーボードパターン（高周波）
    checker_size = 4
    for i in range(80, 120):
        for j in range(70, 110):
            if ((i // checker_size) + (j // checker_size)) % 2 == 0:
                img[i, j] = 0.8
    
    # ノイズを追加
    img += np.random.randn(size, size) * 0.05
    img = np.clip(img, 0, 1)
    
    return img

test_image = create_test_image()

plt.figure(figsize=(8, 8))
plt.imshow(test_image, cmap='gray', vmin=0, vmax=1)
plt.title('テスト画像（様々な特徴を含む）', fontsize=12)
plt.colorbar(label='輝度')
plt.axis('off')
plt.show()

print("特徴:")
print("- 左上: シャープなエッジの四角形")
print("- 右上: 滑らかなエッジの円")
print("- 左下: 斜めの線")
print("- 右下: チェッカーボード（高周波パターン）")
print("- 全体: 緩やかなグラデーション + ノイズ")

---

## 1. フィルタ = 「見たいものを見る眼鏡」

画像フィルタは、特定の特徴を**強調**または**抑制**するための道具です。

In [ ]:
# フィルタの分類

filter_categories = """
╔═══════════════════════════════════════════════════════════════════╗
║                    画像フィルタの分類                             ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  【ローパスフィルタ（平滑化）】                                    ║
║  ├─ 効果: 高周波成分（細かい変化）を除去                          ║
║  ├─ 用途: ノイズ除去、ぼかし                                      ║
║  └─ 例: ボックスフィルタ、ガウシアンフィルタ                      ║
║                                                                   ║
║  【ハイパスフィルタ（微分・エッジ検出）】                          ║
║  ├─ 効果: 低周波成分（滑らかな変化）を除去                        ║
║  ├─ 用途: エッジ検出、輪郭抽出                                    ║
║  └─ 例: Sobel、Prewitt、Laplacian                                 ║
║                                                                   ║
║  【バンドパスフィルタ】                                            ║
║  ├─ 効果: 特定の周波数帯だけを通す                                ║
║  ├─ 用途: 特定サイズの特徴検出                                    ║
║  └─ 例: Difference of Gaussians (DoG)                             ║
║                                                                   ║
╚═══════════════════════════════════════════════════════════════════╝
"""
print(filter_categories)

---

## 2. 平滑化フィルタ（ローパス）

周囲のピクセルの値を**平均**することで、急激な変化（ノイズ、エッジ）を滑らかにします。

### 2.1 ボックスフィルタ（単純平均）

In [ ]:
# ボックスフィルタの定義と効果

def box_filter(size):
    """size×sizeのボックスフィルタ（単純平均）"""
    return np.ones((size, size)) / (size * size)

# 様々なサイズのボックスフィルタ
box_3 = box_filter(3)
box_5 = box_filter(5)
box_9 = box_filter(9)

print("3×3 ボックスフィルタ:")
print(box_3.round(3))
print(f"\n全要素の和: {box_3.sum():.3f}（正規化されている）")

In [ ]:
# ボックスフィルタの効果を可視化

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# 上段：フィルタ適用結果
filtered_images = [
    (test_image, '元画像'),
    (conv2d(test_image, box_3), 'Box 3×3'),
    (conv2d(test_image, box_5), 'Box 5×5'),
    (conv2d(test_image, box_9), 'Box 9×9'),
]

for ax, (img, title) in zip(axes[0], filtered_images):
    ax.imshow(img, cmap='gray', vmin=0, vmax=1)
    ax.set_title(title, fontsize=11)
    ax.axis('off')

# 下段：エッジ部分の拡大
zoom_region = (slice(15, 55), slice(15, 55))  # 四角形のエッジ部分

for ax, (img, title) in zip(axes[1], filtered_images):
    ax.imshow(img[zoom_region], cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'{title}（拡大）', fontsize=11)
    ax.axis('off')

plt.suptitle('ボックスフィルタ：サイズが大きいほどぼかしが強い', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察ポイント:")
print("- エッジが徐々にぼやける")
print("- チェッカーボードパターンが消えていく（高周波の除去）")
print("- ノイズも減少する")

### 2.2 ガウシアンフィルタ（重み付き平均）

ボックスフィルタは全てのピクセルに等しい重みを与えますが、**ガウシアンフィルタ**は中心に近いほど大きな重みを与えます。

In [ ]:
def gaussian_kernel(size, sigma=None):
    """
    ガウシアンカーネルを生成
    
    Parameters:
    -----------
    size : int
        カーネルサイズ（奇数）
    sigma : float, optional
        標準偏差（Noneの場合は size/6 を使用）
    """
    if sigma is None:
        sigma = size / 6
    
    # 座標グリッド
    center = size // 2
    x = np.arange(size) - center
    y = np.arange(size) - center
    X, Y = np.meshgrid(x, y)
    
    # ガウス関数
    kernel = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    
    # 正規化（合計が1になるように）
    kernel = kernel / kernel.sum()
    
    return kernel

# ガウシアンカーネルの例
gauss_5 = gaussian_kernel(5, sigma=1.0)
gauss_9 = gaussian_kernel(9, sigma=1.5)

print("5×5 ガウシアンフィルタ (σ=1.0):")
print(gauss_5.round(4))
print(f"\n全要素の和: {gauss_5.sum():.6f}")

In [ ]:
# ガウシアンカーネルの形状を3D可視化

fig = plt.figure(figsize=(14, 5))

for idx, (kernel, title) in enumerate([
    (box_5, 'ボックス 5×5'),
    (gauss_5, 'ガウシアン 5×5 (σ=1.0)'),
    (gaussian_kernel(9, sigma=2.0), 'ガウシアン 9×9 (σ=2.0)'),
]):
    ax = fig.add_subplot(1, 3, idx+1, projection='3d')
    
    size = kernel.shape[0]
    x = np.arange(size)
    y = np.arange(size)
    X, Y = np.meshgrid(x, y)
    
    ax.plot_surface(X, Y, kernel, cmap='viridis', alpha=0.8)
    ax.set_title(title, fontsize=11)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('重み')

plt.suptitle('カーネルの形状比較（3D）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nガウシアンの利点:")
print("- 中心からの距離に応じた自然な重み付け")
print("- ぼかしの結果がより滑らか（リンギングが少ない）")

In [ ]:
# ボックス vs ガウシアンの比較

fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# 同じサイズのフィルタで比較
box_7 = box_filter(7)
gauss_7 = gaussian_kernel(7, sigma=1.2)

filtered_box = conv2d(test_image, box_7)
filtered_gauss = conv2d(test_image, gauss_7)

# 上段
axes[0, 0].imshow(test_image, cmap='gray', vmin=0, vmax=1)
axes[0, 0].set_title('元画像', fontsize=11)

axes[0, 1].imshow(filtered_box, cmap='gray', vmin=0, vmax=1)
axes[0, 1].set_title('ボックス 7×7', fontsize=11)

axes[0, 2].imshow(filtered_gauss, cmap='gray', vmin=0, vmax=1)
axes[0, 2].set_title('ガウシアン 7×7', fontsize=11)

# 下段：1次元プロファイル（エッジの断面）
row = 35  # 四角形のエッジを横切る行

axes[1, 0].axis('off')
axes[1, 0].text(0.5, 0.5, f'行{row}の\n断面を比較', fontsize=14, ha='center', va='center',
               transform=axes[1, 0].transAxes)

axes[1, 1].plot(test_image[row, :], 'b-', label='元', alpha=0.5)
axes[1, 1].plot(filtered_box[row, :], 'r-', label='ボックス', linewidth=2)
axes[1, 1].set_title('断面比較（ボックス）', fontsize=11)
axes[1, 1].legend()
axes[1, 1].set_xlim(10, 60)
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].plot(test_image[row, :], 'b-', label='元', alpha=0.5)
axes[1, 2].plot(filtered_gauss[row, :], 'g-', label='ガウシアン', linewidth=2)
axes[1, 2].set_title('断面比較（ガウシアン）', fontsize=11)
axes[1, 2].legend()
axes[1, 2].set_xlim(10, 60)
axes[1, 2].grid(True, alpha=0.3)

for ax in axes[0]:
    ax.axhline(y=row, color='red', linestyle='--', alpha=0.5)
    ax.axis('off')

plt.suptitle('ボックス vs ガウシアン：エッジの遷移の違い', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察: ガウシアンの方がエッジの遷移が滑らか")

### 2.3 ノイズ除去の効果を定量評価

In [ ]:
# ノイズ除去の定量評価

# クリーンな画像を作成
clean_image = create_test_image()
np.random.seed(0)  # 再現性のため

# ノイズを追加した画像
noise_levels = [0.05, 0.1, 0.2]

fig, axes = plt.subplots(len(noise_levels), 4, figsize=(16, 12))

for row, noise_std in enumerate(noise_levels):
    noisy = clean_image + np.random.randn(*clean_image.shape) * noise_std
    noisy = np.clip(noisy, 0, 1)
    
    # フィルタ適用
    denoised_box = conv2d(noisy, box_5)
    denoised_gauss = conv2d(noisy, gauss_5)
    
    # PSNR計算
    def psnr(original, processed):
        mse = np.mean((original - processed)**2)
        if mse == 0:
            return float('inf')
        return 10 * np.log10(1.0 / mse)
    
    psnr_noisy = psnr(clean_image, noisy)
    psnr_box = psnr(clean_image, denoised_box)
    psnr_gauss = psnr(clean_image, denoised_gauss)
    
    # 表示
    axes[row, 0].imshow(clean_image, cmap='gray', vmin=0, vmax=1)
    axes[row, 0].set_title('クリーン', fontsize=10)
    
    axes[row, 1].imshow(noisy, cmap='gray', vmin=0, vmax=1)
    axes[row, 1].set_title(f'ノイズ (σ={noise_std})\nPSNR: {psnr_noisy:.1f} dB', fontsize=10)
    
    axes[row, 2].imshow(denoised_box, cmap='gray', vmin=0, vmax=1)
    axes[row, 2].set_title(f'Box 5×5\nPSNR: {psnr_box:.1f} dB', fontsize=10)
    
    axes[row, 3].imshow(denoised_gauss, cmap='gray', vmin=0, vmax=1)
    axes[row, 3].set_title(f'Gaussian 5×5\nPSNR: {psnr_gauss:.1f} dB', fontsize=10)
    
    for ax in axes[row]:
        ax.axis('off')

plt.suptitle('ノイズ除去の効果（PSNR: 高いほど良い）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 3. 微分フィルタ（ハイパス）

画像の**変化**（勾配）を検出するフィルタ。エッジは輝度が急激に変化する場所なので、微分で検出できます。

### 3.1 1次微分：Prewitt、Sobel

1次微分は「隣接ピクセルとの差」を計算します。

In [ ]:
# 1次微分カーネルの定義

# 最も単純な微分（差分）
diff_x = np.array([[-1, 0, 1]])  # 水平方向
diff_y = np.array([[-1], [0], [1]])  # 垂直方向

# Prewitt（単純平均 × 微分）
prewitt_x = np.array([[-1, 0, 1],
                      [-1, 0, 1],
                      [-1, 0, 1]]) / 3
prewitt_y = prewitt_x.T

# Sobel（ガウシアン重み × 微分）
sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]]) / 4
sobel_y = sobel_x.T

print("Sobel-X カーネル:")
print(sobel_x * 4)  # 見やすく4倍して表示
print("\nSobel-Y カーネル:")
print(sobel_y * 4)

print("\n解釈:")
print("- Sobel-X: 左右の差（垂直エッジを検出）")
print("- Sobel-Y: 上下の差（水平エッジを検出）")
print("- 中央行/列の重みが大きい = ガウシアン平滑化の効果")

In [ ]:
# なぜ微分が「エッジ」を検出するのか

fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# 1次元の例（エッジを横切る断面）
x = np.linspace(0, 10, 100)
# ステップエッジ（シグモイド関数で近似）
edge_profile = 1 / (1 + np.exp(-2*(x-5)))
# 1次微分（勾配）
gradient = np.gradient(edge_profile, x)

axes[0, 0].plot(x, edge_profile, 'b-', linewidth=2)
axes[0, 0].set_title('エッジ断面（輝度）', fontsize=11)
axes[0, 0].set_xlabel('位置')
axes[0, 0].set_ylabel('輝度')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(x, gradient, 'r-', linewidth=2)
axes[0, 1].axhline(y=0, color='gray', linestyle='--')
axes[0, 1].set_title('1次微分（勾配）', fontsize=11)
axes[0, 1].set_xlabel('位置')
axes[0, 1].set_ylabel('勾配')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].annotate('ピーク = エッジ位置', xy=(5, gradient.max()), 
                   xytext=(6.5, gradient.max()*0.8),
                   arrowprops=dict(arrowstyle='->', color='red'),
                   fontsize=10, color='red')

# 2次微分
laplacian = np.gradient(gradient, x)
axes[0, 2].plot(x, laplacian, 'g-', linewidth=2)
axes[0, 2].axhline(y=0, color='gray', linestyle='--')
axes[0, 2].set_title('2次微分（ラプラシアン）', fontsize=11)
axes[0, 2].set_xlabel('位置')
axes[0, 2].set_ylabel('2次微分')
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].annotate('ゼロ交差 = エッジ位置', xy=(5, 0), 
                   xytext=(6.5, -0.1),
                   arrowprops=dict(arrowstyle='->', color='green'),
                   fontsize=10, color='green')

# 下段：2D画像での適用
sobel_result_x = conv2d(test_image, sobel_x)
sobel_result_y = conv2d(test_image, sobel_y)
sobel_magnitude = np.sqrt(sobel_result_x**2 + sobel_result_y**2)

axes[1, 0].imshow(sobel_result_x, cmap='RdBu', vmin=-0.5, vmax=0.5)
axes[1, 0].set_title('Sobel-X（垂直エッジ）', fontsize=11)
axes[1, 0].axis('off')

axes[1, 1].imshow(sobel_result_y, cmap='RdBu', vmin=-0.5, vmax=0.5)
axes[1, 1].set_title('Sobel-Y（水平エッジ）', fontsize=11)
axes[1, 1].axis('off')

axes[1, 2].imshow(sobel_magnitude, cmap='gray')
axes[1, 2].set_title('勾配強度 √(Sx² + Sy²)', fontsize=11)
axes[1, 2].axis('off')

plt.suptitle('微分とエッジ検出の原理', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.2 2次微分：Laplacian

In [ ]:
# Laplacianカーネル

# 4近傍ラプラシアン
laplacian_4 = np.array([[0,  1, 0],
                        [1, -4, 1],
                        [0,  1, 0]])

# 8近傍ラプラシアン
laplacian_8 = np.array([[1,  1, 1],
                        [1, -8, 1],
                        [1,  1, 1]])

print("Laplacian カーネル（4近傍）:")
print(laplacian_4)
print(f"\n全要素の和: {laplacian_4.sum()}（ゼロ = DCを保存しない）")

print("\n解釈:")
print("- 中心ピクセルから周囲を引く")
print("- 周囲と同じ値なら出力0、周囲と違えば大きな値")
print("- 方向に依存しない（等方性）")

In [ ]:
# Laplacianの効果

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

lap_result_4 = conv2d(test_image, laplacian_4)
lap_result_8 = conv2d(test_image, laplacian_8)

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('元画像', fontsize=11)

axes[1].imshow(lap_result_4, cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_title('Laplacian（4近傍）', fontsize=11)

axes[2].imshow(lap_result_8, cmap='RdBu', vmin=-1, vmax=1)
axes[2].set_title('Laplacian（8近傍）', fontsize=11)

axes[3].imshow(np.abs(lap_result_8), cmap='gray')
axes[3].set_title('|Laplacian|（エッジ強度）', fontsize=11)

for ax in axes:
    ax.axis('off')

plt.suptitle('Laplacianによるエッジ検出', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察:")
print("- Sobelは方向ごとに検出（X=垂直、Y=水平）")
print("- Laplacianは全方向を一度に検出")
print("- Laplacianはノイズに敏感（2次微分のため）")

---

## 4. 複合フィルタ

基本フィルタを組み合わせて、より高度な効果を実現します。

### 4.1 シャープ化（Unsharp Masking）

**シャープ化 = 元画像 + α × エッジ**

エッジ成分を強調することで、輪郭をくっきりさせます。

In [ ]:
# シャープ化カーネル

def unsharp_mask_kernel(strength=1.0):
    """
    アンシャープマスク用のカーネル
    元画像 - ぼかし = エッジ
    シャープ = 元画像 + strength × エッジ
            = (1+strength)×元画像 - strength×ぼかし
    """
    # 単位カーネル（元画像を保持）
    identity = np.array([[0, 0, 0],
                         [0, 1, 0],
                         [0, 0, 0]])
    
    # ラプラシアン（エッジ検出）
    laplacian = np.array([[0, -1, 0],
                          [-1, 4, -1],
                          [0, -1, 0]])
    
    return identity + strength * laplacian / 4

sharpen_weak = unsharp_mask_kernel(0.5)
sharpen_medium = unsharp_mask_kernel(1.0)
sharpen_strong = unsharp_mask_kernel(2.0)

print("シャープ化カーネル（強度1.0）:")
print(sharpen_medium)
print(f"\n全要素の和: {sharpen_medium.sum():.2f}（1に近い = 全体の明るさを保持）")

In [ ]:
# シャープ化の効果

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

sharpened = [
    (test_image, '元画像'),
    (conv2d(test_image, sharpen_weak), 'シャープ 0.5'),
    (conv2d(test_image, sharpen_medium), 'シャープ 1.0'),
    (conv2d(test_image, sharpen_strong), 'シャープ 2.0'),
]

zoom_region = (slice(15, 55), slice(15, 55))

for idx, (img, title) in enumerate(sharpened):
    img_clipped = np.clip(img, 0, 1)
    
    axes[0, idx].imshow(img_clipped, cmap='gray', vmin=0, vmax=1)
    axes[0, idx].set_title(title, fontsize=11)
    axes[0, idx].axis('off')
    
    axes[1, idx].imshow(img_clipped[zoom_region], cmap='gray', vmin=0, vmax=1)
    axes[1, idx].set_title(f'{title}（拡大）', fontsize=11)
    axes[1, idx].axis('off')

plt.suptitle('シャープ化：エッジ強調の効果', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n観察:")
print("- エッジが強調されてくっきりする")
print("- 強度が高すぎるとハロー（白い縁取り）が発生")
print("- ノイズも強調されてしまう")

### 4.2 Laplacian of Gaussian (LoG)

In [ ]:
def log_kernel(size, sigma):
    """
    Laplacian of Gaussian (LoG) カーネル
    別名：Mexican Hat（メキシカンハット）
    """
    center = size // 2
    x = np.arange(size) - center
    y = np.arange(size) - center
    X, Y = np.meshgrid(x, y)
    
    r2 = X**2 + Y**2
    sigma2 = sigma**2
    
    # LoGの公式
    kernel = -(1 / (np.pi * sigma2**2)) * (1 - r2 / (2 * sigma2)) * np.exp(-r2 / (2 * sigma2))
    
    # 正規化（合計が0になるように）
    kernel = kernel - kernel.mean()
    
    return kernel

# LoGカーネル
log_9 = log_kernel(9, sigma=1.5)

# 3D可視化
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='3d')
x = np.arange(9)
y = np.arange(9)
X, Y = np.meshgrid(x, y)
ax1.plot_surface(X, Y, log_9, cmap='RdBu')
ax1.set_title('LoG カーネル（Mexican Hat）', fontsize=12)

ax2 = fig.add_subplot(122)
ax2.imshow(log_9, cmap='RdBu')
ax2.set_title('LoG カーネル（2D表示）', fontsize=12)
ax2.axis('off')

plt.tight_layout()
plt.show()

print("\nLoGの特徴:")
print("- ガウシアンで平滑化してからラプラシアン（ノイズに強い）")
print("- σでスケールを制御（大きなσ = 大きな特徴を検出）")

### 4.3 Difference of Gaussians (DoG)

In [ ]:
def dog_kernel(size, sigma1, sigma2):
    """
    Difference of Gaussians (DoG) カーネル
    2つの異なるσのガウシアンの差
    """
    g1 = gaussian_kernel(size, sigma1)
    g2 = gaussian_kernel(size, sigma2)
    return g1 - g2

# DoGカーネル
dog = dog_kernel(9, sigma1=1.0, sigma2=2.0)

# LoGとDoGの比較
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# 上段：カーネルの形状
axes[0, 0].imshow(gaussian_kernel(9, 1.0), cmap='Blues')
axes[0, 0].set_title('Gaussian (σ=1.0)', fontsize=11)

axes[0, 1].imshow(gaussian_kernel(9, 2.0), cmap='Blues')
axes[0, 1].set_title('Gaussian (σ=2.0)', fontsize=11)

axes[0, 2].imshow(dog, cmap='RdBu')
axes[0, 2].set_title('DoG = G(1.0) - G(2.0)', fontsize=11)

axes[0, 3].imshow(log_9, cmap='RdBu')
axes[0, 3].set_title('LoG (σ=1.5)', fontsize=11)

# 下段：適用結果
axes[1, 0].imshow(test_image, cmap='gray', vmin=0, vmax=1)
axes[1, 0].set_title('元画像', fontsize=11)

axes[1, 1].imshow(conv2d(test_image, gaussian_kernel(9, 2.0)), cmap='gray', vmin=0, vmax=1)
axes[1, 1].set_title('ぼかし画像', fontsize=11)

dog_result = conv2d(test_image, dog)
axes[1, 2].imshow(dog_result, cmap='RdBu', vmin=-0.2, vmax=0.2)
axes[1, 2].set_title('DoG適用', fontsize=11)

log_result = conv2d(test_image, log_9)
axes[1, 3].imshow(log_result, cmap='RdBu', vmin=-0.05, vmax=0.05)
axes[1, 3].set_title('LoG適用', fontsize=11)

for ax in axes.flatten():
    ax.axis('off')

plt.suptitle('DoG vs LoG：バンドパスフィルタの比較', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nDoGとLoGの関係:")
print("- DoGはLoGの近似（計算が簡単）")
print("- 両者とも特定のスケールの特徴を検出")
print("- SIFT（特徴点検出）ではDoGが使われる")

---

## 5. フィルタの周波数領域での解釈

フィルタの「何を通し、何を止めるか」をFFTで可視化します。

In [ ]:
def visualize_filter_frequency(kernel, name, ax1, ax2):
    """
    カーネルの空間領域と周波数領域を可視化
    """
    # 空間領域
    ax1.imshow(kernel, cmap='RdBu')
    ax1.set_title(f'{name}\n（空間領域）', fontsize=10)
    ax1.axis('off')
    
    # 周波数領域（パディングしてFFT）
    padded = np.zeros((64, 64))
    h, w = kernel.shape
    padded[:h, :w] = kernel
    
    fft = np.fft.fft2(padded)
    fft_shift = np.fft.fftshift(fft)
    magnitude = np.abs(fft_shift)
    
    # 対数スケールで表示
    ax2.imshow(np.log1p(magnitude), cmap='hot')
    ax2.set_title(f'{name}\n（周波数領域）', fontsize=10)
    ax2.axis('off')

# 各種フィルタの周波数特性
fig, axes = plt.subplots(4, 4, figsize=(14, 14))

filters = [
    (box_5, 'ボックス 5×5'),
    (gaussian_kernel(5, 1.0), 'ガウシアン'),
    (sobel_x, 'Sobel-X'),
    (laplacian_8, 'Laplacian'),
]

for idx, (kernel, name) in enumerate(filters):
    visualize_filter_frequency(kernel, name, axes[idx*2//4, idx*2%4], axes[idx*2//4, idx*2%4+1])
    
    # フィルタ適用結果も表示
    result = conv2d(test_image, kernel)
    if 'Sobel' in name or 'Laplacian' in name:
        axes[idx*2//4+1, idx*2%4].imshow(result, cmap='RdBu', vmin=-0.5, vmax=0.5)
    else:
        axes[idx*2//4+1, idx*2%4].imshow(result, cmap='gray', vmin=0, vmax=1)
    axes[idx*2//4+1, idx*2%4].set_title(f'{name}適用', fontsize=10)
    axes[idx*2//4+1, idx*2%4].axis('off')
    
    # 元画像のFFTとフィルタ適用後のFFTの比較
    fft_result = np.fft.fftshift(np.fft.fft2(result))
    axes[idx*2//4+1, idx*2%4+1].imshow(np.log1p(np.abs(fft_result)), cmap='hot')
    axes[idx*2//4+1, idx*2%4+1].set_title('適用後のFFT', fontsize=10)
    axes[idx*2//4+1, idx*2%4+1].axis('off')

plt.suptitle('フィルタの空間領域 vs 周波数領域', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n周波数領域での解釈:")
print("- 中心 = 低周波（ゆっくりした変化）")
print("- 周辺 = 高周波（急激な変化）")
print("- ローパス（ぼかし）: 中心が明るい → 低周波を通す")
print("- ハイパス（エッジ）: 周辺が明るい → 高周波を通す")

---

## 6. 実験：自分だけのフィルタを設計

学んだ知識を使って、オリジナルのフィルタを設計してみましょう。

In [ ]:
# フィルタ設計のテンプレート

def design_filter(kernel, name="カスタムフィルタ"):
    """
    設計したフィルタを評価
    """
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # カーネル表示
    ax1 = axes[0]
    im = ax1.imshow(kernel, cmap='RdBu')
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            ax1.text(j, i, f'{kernel[i,j]:.2f}', ha='center', va='center', fontsize=9)
    ax1.set_title(f'{name}\n合計: {kernel.sum():.2f}', fontsize=11)
    ax1.axis('off')
    
    # 元画像
    axes[1].imshow(test_image, cmap='gray', vmin=0, vmax=1)
    axes[1].set_title('元画像', fontsize=11)
    axes[1].axis('off')
    
    # フィルタ適用
    result = conv2d(test_image, kernel)
    
    # 結果（正規化なし）
    axes[2].imshow(result, cmap='RdBu', vmin=-1, vmax=1)
    axes[2].set_title('適用結果（RdBu）', fontsize=11)
    axes[2].axis('off')
    
    # 結果（絶対値）
    axes[3].imshow(np.abs(result), cmap='gray')
    axes[3].set_title('|適用結果|', fontsize=11)
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return result

In [ ]:
# 例1: 斜め方向のエッジ検出

diagonal_edge = np.array([[-1, 0, 1],
                          [0,  0, 0],
                          [1,  0, -1]])

_ = design_filter(diagonal_edge, "斜めエッジ検出")

In [ ]:
# 例2: エンボス効果

emboss = np.array([[-2, -1, 0],
                   [-1,  1, 1],
                   [0,   1, 2]])

_ = design_filter(emboss, "エンボス効果")

In [ ]:
# 例3: 自分でフィルタを設計してみよう！

# ここに自分のカーネルを定義
my_kernel = np.array([[0,  0, 0],
                      [0,  1, 0],
                      [0,  0, 0]])  # これは単位カーネル（何も変化なし）

# 以下のコメントを外して試してみてください:
# my_kernel = np.array([[-1, -1, -1],
#                       [-1,  9, -1],
#                       [-1, -1, -1]])  # シャープ化

_ = design_filter(my_kernel, "マイフィルタ")

---

## まとめ

In [ ]:
# フィルタ一覧表

summary = """
╔═══════════════════════════════════════════════════════════════════════════════╗
║                       古典的フィルタまとめ                                    ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【平滑化（ローパス）】                                                        ║
║  ├─ ボックス: 単純平均、計算高速、リンギングあり                              ║
║  └─ ガウシアン: 重み付き平均、自然な結果、広く使用                            ║
║                                                                               ║
║  【エッジ検出（ハイパス）】                                                    ║
║  ├─ Sobel/Prewitt: 1次微分、方向別エッジ、勾配計算                           ║
║  └─ Laplacian: 2次微分、全方向エッジ、ノイズに敏感                           ║
║                                                                               ║
║  【複合フィルタ】                                                              ║
║  ├─ シャープ化: 元画像 + エッジ強調                                           ║
║  ├─ LoG: ガウシアン + ラプラシアン（ノイズに強い）                            ║
║  └─ DoG: ガウシアンの差（バンドパス、SIFT）                                   ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【CNNへの接続】                                                               ║
║  • CNNの第1層は、これらの古典フィルタに似たパターンを学習することが多い      ║
║  • 違い: CNNはタスクに最適なフィルタを「自動で」学習する                      ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""
print(summary)

---

## 次のステップ

次のノートブック **85. カーネルの可視化と解釈** では：

- 学習済みCNN（VGG, ResNet）の第1層カーネルを可視化
- 深い層のカーネルの解釈
- 特徴マップの可視化
- 学習前後のカーネルの比較

人間が設計したフィルタと、CNNが学習したフィルタを比較します。